In [4]:
import pandas as pd
import plotly
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import datetime 
import warnings


'C:\\Users\\zpzg\\jp\\pr2'

In [3]:
# os.getcwd()
os.chdir('../pr2')2

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: './pr2'

In [5]:
## data 1 sdot

# os.getcwd()

## read sdot location
# os.chdir('../')
sdot_loc = pd.read_csv('sdot_location.csv',encoding = 'cp949')

os.chdir('./sdot')

sdot_forder = os.listdir('/sdot')
sdot = pd.DataFrame()

# print(sdot_forder)

for files in sdot_forder:
    df = pd.read_csv(files, encoding  = 'cp949')
    sdot = pd.concat([sdot,df])



sdot = sdot[sdot['구분(1:평균/2:최대값/3:최소값)'] == 1]
sdot = sdot[['시리얼','초미세먼지 (㎍/㎥)','미세먼지 (㎍/㎥)','전송시간','초미세먼지 보정 (㎍/㎥)','미세먼지 보정 (㎍/㎥)']]
sdot.columns = ['serial', 'pm25','pm10','trnsmit_time','pm25_fix','pm10_fix']
sdot = sdot[sdot['trnsmit_time'] >= 202107010000]
sdot = sdot[sdot['trnsmit_time'] <= 202107312300]

#sdot data -> sdot_merge
sdot_merge = pd.merge(left = sdot, right = sdot_loc, how = 'inner', on = 'serial')

In [69]:
sdot_merge
# sdot_pm10 = sdot_merge.groupby('gu','trnsmit_time')['pm10'].mean().reset_index()
sdot_pm25 = sdot_merge.groupby(['gu','trnsmit_time'])['pm25'].mean().reset_index()


# sdot_pm25['pm25'] = sdot_pm25['pm25'].round()

# sdot_pm25_fix = sdot_merge.groupby('gu')['pm25_fix'].mean().reset_index()
# sdot_pm25_fix['pm25_fix'] = sdot_pm25_fix['pm25_fix'].round()
# sdot_pm25_fix

sdot_pm25['trnsmit_time'].unique().size


744

In [14]:
os.chdir('../bo')

In [25]:
# 보건환경연구원 7월 평균 데이터 
bo = pd.read_csv('bo.csv', encoding = 'cp949')

# group by gu mean()
#소수점 반올림(간이측정기 평가 기준)
bo_pm25 = bo.groupby('gu')['pm25'].mean().reset_index()
bo_pm25 = bo_pm25.drop([0])
bo_pm25['pm25'] = bo_pm25['pm25'].round()


bo_pm10 = bo.groupby('gu')['pm10'].mean().reset_index()
bo_pm10 = bo_pm10.drop([0])
bo_pm10

,gu,pm10
1,강남구,10.000000
2,강동구,12.375000
3,강북구,9.888889
4,강서구,9.300000
5,관악구,11.900000
6,광진구,12.400000
7,구로구,20.769231
8,금천구,10.769231
9,노원구,22.909091
10,도봉구,9.285714


In [28]:
os.chdir('../gov')

In [41]:
#국가 측정망 
gov = pd.read_csv('gover.csv', encoding = 'cp949')
gov_pm25 = gov[['gu','PM25']]
gov_pm10 = gov[['gu','PM10']]

gov_pm25 = gov_pm25.sort_values('gu').reset_index(drop = True)
gov_pm25


,gu,PM25
0,강남구,16
1,강동구,16
2,강북구,17
3,강서구,17
4,관악구,15
5,광진구,14
6,구로구,12
7,금천구,19
8,노원구,16
9,도봉구,11


In [59]:
## 정확도 계산 시작

# sdot_gover_error

for i in range(0,25):
    sdot25 = sdot_pm25['pm25'][i]
    gov25 = gov_pm25['PM25'][i]
    error_sdot = abs((sdot25-gov25)/gov25)
    print(((1-error_sdot)*100).round(1))

print("-------------------------------------------------------")

    
    
for i in range(0,25):
    sdot25 = sdot_pm25_fix['pm25_fix'][i]
    gov25 = gov_pm25['PM25'][i]
    error_sdot = abs((sdot25-gov25)/gov25)
    print(((1-error_sdot)*100).round(1))
    


81.2
68.8
64.7
76.5
93.3
92.9
91.7
84.2
68.8
100.0
76.5
86.7
92.9
85.7
81.8
88.2
91.7
73.3
93.8
82.4
81.2
100.0
75.0
82.4
75.0
-------------------------------------------------------
81.2
75.0
70.6
88.2
93.3
92.9
91.7
84.2
75.0
90.9
82.4
86.7
92.9
92.9
81.8
76.5
100.0
80.0
93.8
88.2
81.2
100.0
81.2
82.4
75.0
